# Set-up

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain
import methods
import pickle
import warnings
import plotly as py
from data_extraction_methods import get_openpose_output, determine_video_meta_data, get_period_person_division, get_person_period_division

warnings.filterwarnings('ignore')

In [ ]:
% matplotlib inline

## Clip name

In [ ]:
clip_name = 'jeroenkrol_28121995_184_80_16500.mp4'
image_h, image_w, fps = determine_video_meta_data('C:\\Users\\jaspe\\Notebooks\\Github\\JADS-2D-pose-estimation\\videos\\{}'.format(clip_name))
people_per_file = get_openpose_output('C:\\Users\\jaspe\\Notebooks\\Github\\JADS-2D-pose-estimation\\coordinates\\{}'.format(clip_name))

## Load prepared data

In [ ]:
# plottables_per_file, period_person_division = methods.\
#             get_plottables_per_file_and_period_person_division(people_per_file, fps, connections)

period_person_division = get_period_person_division(people_per_file, fps)    
#plottables_per_file = methods.get_plottables_per_file(people_per_file, connections)
    
    
person_period_division = get_person_period_division(period_person_division)

mean_x_per_person = methods.get_mean_x_per_person(person_period_division)

normalized_moved_distance_per_person = methods.normalize_moved_distance_per_person(mean_x_per_person)

# Only include identified people that move more than a set movement threshold
maximum_normalized_distance = max(normalized_moved_distance_per_person.values())
movement_threshold = maximum_normalized_distance / 4
moving_people = [key for key, value in normalized_moved_distance_per_person.items() if value > movement_threshold]

period_running_person_division_df = methods.get_period_running_person_division_df(mean_x_per_person, moving_people)

dbscan_subsets = methods.get_dbscan_subsets(maximum_normalized_distance, period_running_person_division_df)

max_dbscan_subset = dbscan_subsets[
    np.argmax([sum([len(person_period_division[person]) for person in subset]) for subset in dbscan_subsets])]

running_person_identifiers = methods.determine_running_person_identifiers(period_running_person_division_df, 
                                                      max_dbscan_subset, 
                                                      maximum_normalized_distance*4,
                                                      maximum_normalized_distance**2)

running_fragments, turning_fragments, fragments = \
            methods.get_running_and_turning_fragments(running_person_identifiers, mean_x_per_person, period_running_person_division_df, moving_people,
                                                     fps, plot=True)
    
period_running_person_division = {period:{person: coords for person,
                      coords in period_dictionary.items() if person in running_person_identifiers}
                     for period, period_dictionary in period_person_division.items()}
period_running_person_division = dict(filter(lambda x: x[1] != {}, period_running_person_division.items()))
    
coord_df = methods.get_dataframe_from_coords(period_person_division,
                                               running_person_identifiers,
                                               running_fragments)

# rotation_angle = methods.get_rotation_angle(coord_df)

# period_person_division = {
# period: {person: np.array([methods.rotate((x, y), rotation_angle) + (z,) for x, y, z in coords])
#          for person, coords in period_dictionary.items()}
# for period, period_dictionary in period_person_division.items()}

period_running_person_division, running_plottables, turning_plottables = \
    methods.get_plottables(period_person_division, running_person_identifiers, running_fragments, turning_fragments)

# Plotting

In [ ]:
% matplotlib notebook

In [ ]:
methods.plot_person(running_plottables, image_h, image_w, zoom=True, pad=3, sleep=0.05)

In [ ]:
# coord_list = methods.prepare_data_for_plotting(period_person_division, running_person_identifiers, running_fragments)
# coord_df = methods.get_dataframe_from_coords(coord_list)
# feature_df, coord_df = methods.to_feature_df(coord_df)
# feature_df = methods.forward_leaning(feature_df, coord_df)
# feature_df.head()